In [10]:
import pandas as pd
from miditok.pytorch_data import DatasetMIDI
from pathlib import Path
import torch
from miditok import REMI

k=1
train_data=pd.read_csv(f'../train data/k-folds_giant-midi_type0/giant-midi_type0_train_set_k{k}.csv')
validation_data=pd.read_csv(f'../train data/k-folds_giant-midi_type0/giant-midi_type0_test_set_k{k}.csv')

#train_data=pd.read_csv('../train data/right_hand_train_set.csv')
#validation_data=pd.read_csv('../train data/right_hand_validation_set.csv')
#test_data=pd.read_csv('../train data/right_hand_test_set.csv')

train_data['type0_path']=train_data['type0_path'].apply(lambda x: x.replace('../author-profiling-in-symbolic-music/','../train data/'))
train_data['type0_path']=train_data['type0_path'].apply(lambda x: x.replace('../../train data/k-folds_giant-midi_type0/','../train data/giant midi/'))

validation_data['type0_path']=validation_data['type0_path'].apply(lambda x: x.replace('../author-profiling-in-symbolic-music/','../train data/'))
validation_data['type0_path']=validation_data['type0_path'].apply(lambda x: x.replace('../../train data/k-folds_giant-midi_type0/','../train data/giant midi/'))

In [17]:
validation_data['type0_path']=validation_data['type0_path'].apply(lambda x: x.replace('../../','../'))

In [11]:
scores_df=validation_data
paths_column_name='type0_path'

In [18]:
# Define a custom DatasetMIDI subclass
class CustomDatasetMIDI(DatasetMIDI):
    def __init__(self, files_paths, labels, tokenizer, max_seq_len, bos_token_id, eos_token_id):
        super().__init__(files_paths, tokenizer, max_seq_len, bos_token_id, eos_token_id)
        self.labels = labels

    def __getitem__(self, idx):
        item = super().__getitem__(idx)
        item["labels"] = self.labels[idx]  # Add labels to the item dictionary
        return item
  
midi_paths = []
labels=[]
paths=scores_df[paths_column_name]

#paths=paths.apply(lambda x: x.replace('../../','../'))

for i,score in enumerate(paths):

  midi_paths.append(Path(score))

  integer_label= 0 if scores_df['composer_gender'][i] == 'Male' else 1

  labels.append(torch.tensor(integer_label))  # Modify this line to extract the label

# Initialize the tokenizer
tokenizer = REMI.from_pretrained("Natooz/Maestro-REMI-bpe20k")


# Initialize the dataset
dataset = CustomDatasetMIDI(
  files_paths=midi_paths,
  labels=labels,  # Pass the labels to the dataset
  tokenizer=tokenizer,
  max_seq_len=1024,
  bos_token_id=tokenizer.pad_token_id,
  eos_token_id=tokenizer["BOS_None"]
)

In [ ]:
error_indices = []

for i, sequence in enumerate(range(len(dataset))):
    try:
        1+1
        print(i, dataset[sequence]['input_ids'])
    except Exception as e:
        print(f"Error at index {i}: {e}")
        error_indices.append(i)

print("Indices with errors:", error_indices)

0 tensor([ 246, 1739,  788,  ..., 1428, 1291, 1280])
1 tensor([  175, 13897,  8979,  ...,   434,    37,   434])
2 tensor([3561, 1662, 2358,  ..., 2734,   55, 1799])
3 tensor([  626,  8861, 13341,  ...,  2006,  1716,  4541])
4 tensor([ 3561,   196,  4789,  ..., 14924,  2257,   326])
5 tensor([ 2718,   256, 13557,  ...,   154,  1426,  1428])
6 tensor([  175,  2141,  2101,   134,  7102,  6824,   138,  1658,   142,  3084,
          146,  3210,  3431,   150,  1658,   154,  2141,  2101,  3296,   158,
         7102,  6824,   175,  2011,  1616,   134,  2666,  2950,   138,  7102,
         6824,   142,  1658,  1694,   146,  3084,  1616,   150,  1822,  1356,
         2863,   214,  1947,  1533,  2251,   458,  3785,   175,  2011,  2141,
          134,  2101,   138,  2724,  2617,  3210,  2101,   142,  2645,  1533,
          146,  1761,  2617,  3210,  1283,  3356,   458,  2484,   154,  2617,
         3210,  3747,   158,  2783,  1291,   175,  1658,  3210,  2645,   134,
         1546,  1520,  1376,   1